<a href="https://colab.research.google.com/github/RawanKazma/Helmet_Detection/blob/main/HatOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Collect and Prepare Data


In [3]:
json_filepath_test = '/content/drive/MyDrive/Dataset/test/_annotations.coco.json'
imgs_filepath_test = '/content/drive/MyDrive/Dataset/test'

json_filepath_train = '/content/drive/MyDrive/Dataset/train/_annotations.coco.json'
imgs_filepath_train = '/content/drive/MyDrive/Dataset/train/'

In [31]:
# read the file
with open(json_filepath_train, 'r') as file:
  annotations_data = json.load(file)

with open(json_filepath_test, 'r') as file:
  annotations_data_test = json.load(file)

#print(json.dumps(annotations_data, indent=2))
annotations = annotations_data.get('annotations', [])
annotations_test = annotations_data_test.get('annotations', [])
#print(annotations)


# Extract cats
categories = annotations_data['categories']
print(categories)

#2 is true else is false
dict_labels_binary = {category['id']: 0 if category['name'] != 'helmet' else 1 for category in categories}
print(dict_labels_binary)



[{'id': 0, 'name': 'Workers', 'supercategory': 'none'}, {'id': 1, 'name': 'head', 'supercategory': 'Workers'}, {'id': 2, 'name': 'helmet', 'supercategory': 'Workers'}, {'id': 3, 'name': 'person', 'supercategory': 'Workers'}]
{0: 0, 1: 0, 2: 1, 3: 0}


In [5]:
# Add binary labels to each annotation
#for annotation in annotations:
 #   category_id = annotation.get('category_id', -1)
  #  annotation['category_id'] = dict_labels_binary.get(category_id, -1)

#inspect
for annotation in annotations:
  print(annotation)





#note:[x, y, width, heightt ex: bbox starts at (214, 198) and has a width of 19 pixels and a height of 22 pixels.

Streaming output truncated to the last 5000 lines.
{'id': 17550, 'image_id': 3425, 'category_id': 2, 'bbox': [41, 124, 39, 52], 'area': 2028, 'segmentation': [], 'iscrowd': 0}
{'id': 17551, 'image_id': 3425, 'category_id': 2, 'bbox': [232, 180, 29, 37], 'area': 1073, 'segmentation': [], 'iscrowd': 0}
{'id': 17552, 'image_id': 3425, 'category_id': 2, 'bbox': [145, 193, 9, 14], 'area': 126, 'segmentation': [], 'iscrowd': 0}
{'id': 17553, 'image_id': 3425, 'category_id': 2, 'bbox': [81, 183, 10, 17], 'area': 170, 'segmentation': [], 'iscrowd': 0}
{'id': 17554, 'image_id': 3425, 'category_id': 2, 'bbox': [336, 0, 36, 13], 'area': 468, 'segmentation': [], 'iscrowd': 0}
{'id': 17555, 'image_id': 3425, 'category_id': 2, 'bbox': [40, 0, 40, 15], 'area': 600, 'segmentation': [], 'iscrowd': 0}
{'id': 17556, 'image_id': 3426, 'category_id': 2, 'bbox': [46, 101, 36, 33], 'area': 1188, 'segmentation': [], 'iscrowd': 0}
{'id': 17557, 'image_id': 3426, 'category_id': 2, 'bbox': [93, 107, 23, 30], 'ar

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
import os

# Specify the path for the test dataset within Colab's writable space
train_dataset_dir_colab = '/content/drive/MyDrive/Data/Train'
test_dataset_dir_colab = '/content/drive/MyDrive/Data/Test'


# Specify the class names
class_names = ['without_hard_hat', 'with_hard_hat']

# Construct paths for 'without_hard_hat' and 'with_hard_hat' directories within Colab
without_hat_dir_colab_train = os.path.join(train_dataset_dir_colab, class_names[0])
with_hat_dir_colab_train = os.path.join(train_dataset_dir_colab, class_names[1])

# Construct paths for 'without_hard_hat' and 'with_hard_hat' directories within Colab
without_hat_dir_colab_test = os.path.join(test_dataset_dir_colab, class_names[0])
with_hat_dir_colab_test = os.path.join(test_dataset_dir_colab, class_names[1])

# Create the directories if they don't exist
os.makedirs(without_hat_dir_colab_train, exist_ok=True)
os.makedirs(with_hat_dir_colab_train, exist_ok=True)
os.makedirs(without_hat_dir_colab_test, exist_ok=True)
os.makedirs(with_hat_dir_colab_test, exist_ok=True)


# Separate Images in Sub folders

In [9]:
import os
import shutil

source_photos_dir = imgs_filepath_train
images = annotations_data.get('images', [])
processed_image_ids = []

for image in images:
    Image_name = image['file_name']
    image_path = os.path.join(source_photos_dir, Image_name)
    image_id = image['id']

    # Find the annotations for the current image_id in the list
    matching_annotations = [ann for ann in annotations if ann.get('image_id') == image_id]

    # Check if there are matching annotations
    if matching_annotations:
        category_ids = []
        for annotation in matching_annotations:
            category_id = annotation.get('category_id', -1)
            category_ids.append(category_id)
            #print(category_id,annotation)

        # Check if the list has at least one occurrence of value 2
        if 2 in category_ids:
            #print(image_id, 'HAT', category_id)
            destination_path = os.path.join(with_hat_dir_colab_train, Image_name)
            shutil.copy(image_path, destination_path)

        else:
          #print(image_id, '----------------',category_id)
          destination_path = os.path.join(without_hat_dir_colab_train, Image_name)
          shutil.copy(image_path, destination_path)
    else:
        print(f"No annotations found for {Image_name}.")


Streaming output truncated to the last 5000 lines.
2 {'id': 23260, 'image_id': 4565, 'category_id': 2, 'bbox': [367, 69, 47, 50], 'area': 2350, 'segmentation': [], 'iscrowd': 0}
[2, 2, 2]
4565 HAT 2
1 {'id': 23261, 'image_id': 4566, 'category_id': 1, 'bbox': [305, 138, 29, 37], 'area': 1073, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23262, 'image_id': 4566, 'category_id': 1, 'bbox': [125, 117, 38, 42], 'area': 1596, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23263, 'image_id': 4566, 'category_id': 1, 'bbox': [258, 131, 37, 40], 'area': 1480, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23264, 'image_id': 4566, 'category_id': 1, 'bbox': [88, 142, 31, 33], 'area': 1023, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23265, 'image_id': 4566, 'category_id': 1, 'bbox': [154, 146, 26, 26], 'area': 676, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23266, 'image_id': 4566, 'category_id': 1, 'bbox': [392, 137, 22, 38], 'area': 836, 'segmentation': [], 'iscrowd': 0}
1 {'id': 23267, 'image_id': 4566, 'c

In [32]:

source_photos_dir = imgs_filepath_test
images = annotations_data_test.get('images', [])
processed_image_ids = []

for image in images:
    Image_name = image['file_name']
    image_path = os.path.join(source_photos_dir, Image_name)
    image_id = image['id']

    # Find the annotations for the current image_id in the list
    matching_annotations = [ann for ann in annotations_test if ann.get('image_id') == image_id]

    # Check if there are matching annotations
    if matching_annotations:
        category_ids = []
        for annotation in matching_annotations:
            category_id = annotation.get('category_id', -1)
            category_ids.append(category_id)
            print(category_id,annotation)
        print(category_ids)

        if 2 in category_ids:
            print(image_id, 'HAT', category_id)
            destination_path = os.path.join(with_hat_dir_colab_test, Image_name)
            shutil.copy(image_path, destination_path)
        else:
          print(image_id, '----------------',category_id)
          destination_path = os.path.join(without_hat_dir_colab_test, Image_name)
          shutil.copy(image_path, destination_path)
    else:
        print(f"No annotations found for {Image_name}.")


Streaming output truncated to the last 5000 lines.
[2]
1062 HAT 2
2 {'id': 5323, 'image_id': 1063, 'category_id': 2, 'bbox': [79, 142, 35, 44], 'area': 1540, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5324, 'image_id': 1063, 'category_id': 2, 'bbox': [169, 137, 35, 44], 'area': 1540, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5325, 'image_id': 1063, 'category_id': 2, 'bbox': [232, 145, 29, 44], 'area': 1276, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5326, 'image_id': 1063, 'category_id': 2, 'bbox': [256, 134, 39, 51], 'area': 1989, 'segmentation': [], 'iscrowd': 0}
[2, 2, 2, 2]
1063 HAT 2
2 {'id': 5327, 'image_id': 1064, 'category_id': 2, 'bbox': [207, 58, 71, 74], 'area': 5254, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5328, 'image_id': 1064, 'category_id': 2, 'bbox': [189, 53, 68, 24], 'area': 1632, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5329, 'image_id': 1064, 'category_id': 2, 'bbox': [206, 0, 72, 47], 'area': 3384, 'segmentation': [], 'iscrowd': 0}
2 {'id': 5330, 'image_id':

# preprocessing Data



*   Utilize ImageDataGenerator to preprocess and augment the training images.
*   Create separate generators for training and validation data.

*  Flow the images in batches using generators with specified augmentation parameters.











In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image data generators with preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Assuming it's binary classification
)

# Flow validation images in batches using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/Test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Assuming it's binary classification
)


Found 5269 images belonging to 2 classes.
Found 1766 images belonging to 2 classes.


# Model Architecture
*  Use the EfficientNetB0 architecture as a base model.
*   Add a Global Average Pooling layer and a Dense layer with a sigmoid activation for binary classification.

* Compile the model with the Adam optimizer and binary crossentropy loss.

In [34]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

# Create and compile the model
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


16705208/16705208 [==============================] - 1s 0us/step


# Model Training

*  Train the model using the training data generator.
*  Specify the number of epochs and provide the validation data generator for evaluation during training.




In [ ]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
165/165 [==============================] - 771s 5s/step - loss: 0.3013 - accuracy: 0.9114 - val_loss: 0.3159 - val_accuracy: 0.9083
Epoch 2/10
165/165 [==============================] - 743s 5s/step - loss: 0.2878 - accuracy: 0.9171 - val_loss: 0.3108 - val_accuracy: 0.9083
Epoch 3/10
165/165 [==============================] - 731s 4s/step - loss: 0.2881 - accuracy: 0.9171 - val_loss: 0.3070 - val_accuracy: 0.9083
Epoch 4/10
165/165 [==============================] - 723s 4s/step - loss: 0.2886 - accuracy: 0.9171 - val_loss: 0.3074 - val_accuracy: 0.9083
Epoch 5/10
159/165 [===========================>..] - ETA: 18s - loss: 0.2859 - accuracy: 0.9179